In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import time
import re
import os

In [7]:
def int_to_month(month):
    if month == 1:
        month = 'Январь'
    elif month == 2:
        month = 'Февраль'
    elif month == 3:
        month = 'Март'
    elif month == 4:
        month = 'Апрель'
    elif month == 5:
        month = 'Май'
    elif month == 6:
        month = 'Июнь'
    elif month == 7:
        month = 'Июль'
    elif month == 8:
        month = 'Август'
    elif month == 9:
        month = 'Сентябрь'
    elif month == 10:
        month = 'Октябрь'
    elif month == 11:
        month = 'Ноябрь'
    elif month == 12:
        month = 'Декабрь'
    return month

def splitdate(date):
    date_split = date.split('.')
    day = int(date_split[0])
    month = int(date_split[1])
    strmonth = int_to_month(month)
    if month == 1:
        month = 'января'
    elif month == 2:
        month = 'февраля'
    elif month == 3:
        month = 'марта'
    elif month == 4:
        month = 'апреля'
    elif month == 5:
        month = 'мая'
    elif month == 6:
        month = 'июня'
    elif month == 7:
        month = 'июля'
    elif month == 8:
        month = 'августа'
    elif month == 9:
        month = 'сентября'
    elif month == 10:
        month = 'октября'
    elif month == 11:
        month = 'ноября'
    elif month == 12:
        month = 'декабря'
    year = int(date_split[2]) if len(date_split) > 2 else None        
    return day, month, year, strmonth

def get_result(score1, score2):
    if score1 > score2:
        result1 = 'Победа'
        result2 = 'Поражение'
    elif score2 > score1:
        result2 = 'Победа'
        result1 = 'Поражение'
    else:
        result1 = result2 = 'Ничья'
    return result1, result2

def get_idteam(team, teams_dict):
    id_team = teams_dict.get(team)
    if id_team is None:
        teams_dict['COUNTER'] += 1
        id_team = teams_dict['COUNTER']
        teams_dict[team] = id_team
    return id_team

def get_idplayer(player, players_dict):
    id_player = players_dict.get(player)
    if id_player is None:
        players_dict['COUNTER'] += 1
        id_player = players_dict['COUNTER']
        players_dict[player] = id_player
    return id_player

def get_playerinfo_dict(player, shortname, player_link, playersinfo_dict):
    info = playersinfo_dict.get(shortname)
    if info is None:
        driver.get(player_link)
        time.sleep(TIME_SLEEP)
        player_soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        name = player_soup.find('div', class_='teamHeader__name').contents[0].strip()
        
        try:
            role = player_soup.find('div', class_='teamHeader__info--player-type-name').string
            if role is None:
                role = player_soup.find('div', class_='teamHeader__info--player-type-name').contents[0].strip()[:-2]
        except:
            role = np.nan
        
        try:
            bd = re.search('\(\S+\)', player_soup.find('span', class_='jsl-age').string)[0][1:-1]
            bd_day, bd_month, bd_year, bd_strmonth = splitdate(bd)
            bd_date = str(bd_day) + ' ' + bd_month + ' ' + str(bd_year)
        except:
            bd_date, bd_strmonth = np.nan, np.nan
        playersinfo_dict[shortname] = name, role, bd_date, bd_strmonth
    else:
        name, role, bd_date, bd_strmonth = info
    return name, role, bd_date, bd_strmonth

def get_playertime(main, time):
    if main == 'Старт':
        starttime = 0
        endtime = time if time else 90
    else:
        if time:
            if type(time) == int:
                starttime = time
                endtime = 90
            else:
                starttime = time[0]
                endtime = time[1]
        else:
            starttime = np.nan
            endtime = np.nan
    playtime = endtime - starttime
    return starttime, endtime, playtime

def get_time_event(main, events):
    goals, assists, penalties, yc, rc, autogoals = 0, 0, 0, 0, 0, 0
    time = None
    for event in events:
        event_type = event.span.get('class')[1]
        if event_type == 'substitution-in':
            time = int(re.search('\d+', event.get('title'))[0])
        elif event_type == 'substitution-out':
            if time is None:
                time = int(re.search('\d+', event.get('title'))[0])
            else:
                time = (time, int(re.search('\d+', event.get('title'))[0]))
        elif event_type == 'soccer-ball':
            goals += 1
            if event.next_sibling is not None and event.next_sibling.string == '(Пенальти)':
                penalties += 1
        elif event_type == 'soccer-ball-own':
            autogoals += 1
        elif event_type == 'y-card':
            yc += 1
        elif event_type == 'r-card':
            rc += 1
        elif event_type == 'yr-card':
            yc += 1
            rc += 1
    starttime, endtime, playtime = get_playertime(main, time)
    return goals, assists, penalties, yc, rc, autogoals, starttime, endtime, playtime
    

def get_playerinfo(player, main, players_dict, playersinfo_dict, assists_dict):
    number = int(player.div.string)
    player_country = player.span.get('title')
    
    shortname = player.a.string
    if shortname is None:
        shortname = player.a.contents[0]
    shortname = shortname.strip()
    
    link = re.search('/\S+/', player.a.get('onclick'))
    if link is None:
        return [np.nan, number, np.nan, shortname] + [np.nan] * 13 + [player_country]
    player_link = flashscore + link[0]
    name, role, bd_date, bd_strmonth = get_playerinfo_dict(player, shortname, player_link, playersinfo_dict)
    
    id_player = get_idplayer(name, players_dict)
    
    events = player('div', class_='icon-lineup')
    
    goals, assists, penalties, yc, rc, autogoals, starttime, endtime, playtime = get_time_event(main, events)
    
    assists = assists_dict.get(shortname, 0)
        
    return [role, number, name, shortname, id_player, starttime, endtime, playtime,
            goals, assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country]

def get_coachinfo(player, players_dict, playersinfo_dict):
    number = np.nan
    player_country = player.span.get('title')
    
    shortname = player.a.string
    if shortname is None:
        shortname = player.a.contents[0]
    shortname = shortname.strip()
    
    link = re.search('/\S+/', player.a.get('onclick'))
    if link is None:
        return [np.nan, number, np.nan, shortname] + [np.nan] * 13 + [player_country]
    player_link = flashscore + link[0]
    name, role, bd_date, bd_strmonth = get_playerinfo_dict(player, shortname, player_link, playersinfo_dict)
    
    id_player = get_idplayer(name, players_dict)
    
    goals, assists, penalties, yc, rc, autogoals, starttime, endtime, playtime = [np.nan] * 9
        
    return (role, number, name, shortname, id_player, starttime, endtime, playtime,
            goals, assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country)

In [8]:
def scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(1)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [9]:
stat_columns = ['Владение мячом', 'Удары', 'Удары в створ', 'Удары мимо', 'Блок-но ударов', 'Штрафные', 'Угловые', 'Офсайды',
               'Сэйвы', 'Фолы', 'Красные карточки', 'Желтые карточки', 'Всего передач', 'Oтборы', 'Атаки', 'Опасные атаки']
columns = ['страна', 'турнир', 'сезон', 'тур', 'месяц матча', 'матч', 'хозяева', 'хозяева голы', 'гости голы', 'гости',
           'гости\\хозяева', 'результат', 'id_матча', 'id_результата', 'амплуа', 'номер игрока', 'фио', 'фио кратко', 'id_фио',
           'минута выхода', 'минута ухода', 'минут на поле', 'гол', 'ассист','жк', 'кк',
           'пенальти', 'автогол', 'ссылка игрока', 'дата рождения', 'месяц рождения', 'страна игрока', 'старт\\запас',
           'команда', 'id_команды', 'CategoryColumn', 'судья', 'стадион'] + stat_columns
TIME_SLEEP = 2

In [21]:
def parse(season, url, driver, df_old):

    if df_old is not None:
        id_match = df_old['id_матча'].max() + 1
        
        teams_dict = {'COUNTER': df_old['id_команды'].max()}
    
        players_dict = {'COUNTER': df_old['id_фио'].max()}
        players_dict.update({player: id_player for player, id_player in zip(df_old['фио'], df_old['id_фио'])})
        
        df_old = df_old[(df_old['страна'] == country) & (df_old['турнир'] == tournament) & (df_old['сезон'] == season)]
        
        teams_dict.update({team: id_team for team, id_team in zip(df_old['команда'], df_old['id_команды'])})
        playersinfo_dict = {player: [name, role, bd, bd_month] for player, name, role, bd, bd_month
                            in zip(df_old['фио кратко'], df_old['фио'], df_old['амплуа'],
                                   df_old['дата рождения'], df_old['месяц рождения'])}
    else:
        id_match = 1
        teams_dict = {'COUNTER': 0}
        players_dict = {'COUNTER': 0}
        playersinfo_dict = {}

    driver.get(flashscore + url)
    time.sleep(TIME_SLEEP)
    scroll(driver)

    while True:
        try:
            driver.find_element_by_class_name('event__more').click()
            time.sleep(TIME_SLEEP)
            scroll(driver)
        except:
            break
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    matches = soup.find('div', class_='event__header').find_next_siblings('div')

    n = 0
    df = pd.DataFrame(columns=columns)

    for match in tqdm(matches):

        if match.get('class')[0] == 'event__round':
            tour = int(re.search('\d+', match.string)[0])
            continue

        team1_name = match.find('div', class_='event__participant--home').string
        if team1_name is None:
            team1_name = match.find('div', class_='event__participant--home').contents[-1]
        team1_name = team1_name.strip()
        team2_name = match.find('div', class_='event__participant--away').string
        if team2_name is None:
            team2_name = match.find('div', class_='event__participant--away').contents[0]
        team2_name = team2_name.strip()
        
        if match.find('div', class_='event__stage') is not None:
            df = df.append(pd.DataFrame([[country, tournament, season, tour, 'ТП', 'ТП', team1_name, 'ТП', 'ТП',
                               team2_name] + [np.nan] * 44], columns=columns))
            continue
            
        span = match.find('div', class_='event__scores').span
        score1 = int(span.string)
        score2 = int(span.find_next('span').string)
        score = str(score1) + ' : ' + str(score2)
        match_name = team1_name + ' ' + score + ' ' + team2_name

        if df_old is not None and (country + tournament + season + match_name == \
                                   df_old['страна'] + df_old['турнир'] + df_old['сезон'] + df_old['матч']).any():
            continue

        while True:
            try:

                n = 0
                df_match = pd.DataFrame(columns=columns)

                result1, result2 = get_result(score1, score2)

                date = match.find('div', class_='event__time').string[:5]
                _, _, _, month = splitdate(date)

                id_team1 = get_idteam(team1_name, teams_dict)
                id_team2 = get_idteam(team2_name, teams_dict)

                match_link = flashscore + '/match/' + match.get('id')[4:]    
                driver.get(match_link)

                while True:
                    try:
                        driver.find_element_by_link_text('Статистика').click()
                        break
                    except:
                        time.sleep(TIME_SLEEP)
                time.sleep(TIME_SLEEP)
                match_soup = BeautifulSoup(driver.page_source, 'html.parser')

                team1_stats_dict = {stat: np.nan for stat in stat_columns}
                team2_stats_dict = {stat: np.nan for stat in stat_columns}
                stats = match_soup.find('div', class_='statContent')
                if stats is None:
                    time.sleep(1)
                    match_soup = BeautifulSoup(driver.page_source, 'html.parser')
                for stat in stats('div', class_='statRow'):
                    stat_name = stat.find('div', class_='statText statText--titleValue').string
                    if stat_name not in stat_columns:
                        continue
                    team1_stat = stat.find('div', class_='statText statText--homeValue').string
                    team2_stat = stat.find('div', class_='statText statText--awayValue').string
                    try:
                        team1_stats_dict[stat_name] = int(team1_stat)
                        team2_stats_dict[stat_name] = int(team2_stat)
                    except:
                        team1_stats_dict[stat_name] = team1_stat
                        team2_stats_dict[stat_name] = team2_stat
                team1_stats = list(team1_stats_dict.values())
                team2_stats = list(team2_stats_dict.values())

                while True:
                    try:
                        driver.find_element_by_link_text('Составы').click()
                        break
                    except:
                        time.sleep(TIME_SLEEP)
                time.sleep(TIME_SLEEP)
                match_soup = BeautifulSoup(driver.page_source, 'html.parser')

                referee_name = np.nan
                stadium = np.nan
                referee_stadium = match_soup('div', class_='content')
                for content in referee_stadium:
                    if 'Судья' in content.string:
                        referee_name = content.string.strip()[7:]
                    elif 'Стадион' in content.string:
                        stadium = content.string.strip()[9:]

                assists_dict = {}
                for assistant in match_soup('span', class_='assist note-name'):
                    assistant_name = assistant.a.string.strip()
                    assists = assists_dict.get(assistant_name, 0)
                    assists += 1
                    assists_dict[assistant_name] = assists

                table = match_soup.find('table', class_='parts')
                double_players = table('tr')

                for double_player in double_players:

                    cast = double_player.find('td', colspan='2')
                    if cast is not None:
                        main = 'Старт' if cast.string == 'Стартовые составы' else 'Запас'
                        continue

                    team1_player = double_player.find('td', class_='summary-vertical fl')
                    if team1_player.a is not None:
                        role, number, name, shortname, id_player, starttime, endtime, playtime, goals, \
                        assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country = \
                        get_playerinfo(team1_player, main, players_dict, playersinfo_dict, assists_dict)

                        df_match.loc[n] = [country, tournament, season, tour, month, match_name, team1_name, score1, score2,
                                        team2_name, 'Хозяева', result1, id_match, 2*id_match - 1, role, number, name, shortname,
                                        id_player, starttime, endtime, playtime, goals, assists, yc, rc,
                                        penalties, autogoals, player_link, bd_date, bd_strmonth, player_country, main, team1_name,
                                        id_team1, match_link, referee_name, stadium] + team1_stats
                        n += 1

                    team2_player = double_player.find('td', class_='summary-vertical fr')
                    if team2_player.a is not None:
                        role, number, name, shortname, id_player, starttime, endtime, playtime, goals, \
                        assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country = \
                        get_playerinfo(team2_player, main, players_dict, playersinfo_dict, assists_dict)

                        df_match.loc[n] = [country, tournament, season, tour, month, match_name, team1_name, score1, score2,
                                        team2_name, 'Гости', result2, id_match, 2*id_match, role, number, name, shortname,
                                        id_player, starttime, endtime, playtime, goals, assists, yc, rc,
                                        penalties, autogoals, player_link, bd_date, bd_strmonth, player_country, main, team2_name,
                                        id_team2, match_link, referee_name, stadium] + team2_stats
                        n += 1

                    #break

                double_coaches = match_soup.find('div', class_='coaches-wrapper')

                team1_coach = double_coaches.find('td', class_='summary-vertical fl')
                if team1_coach.a is not None:
                    role, number, name, shortname, id_player, starttime, endtime, playtime, goals, \
                    assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country = \
                    get_coachinfo(team1_coach, players_dict, playersinfo_dict)

                    df_match.loc[n] = [country, tournament, season, tour, month, match_name, team1_name, score1, score2,
                                        team2_name, 'Хозяева', result1, id_match, 2*id_match - 1, role, number, name, shortname,
                                        id_player, starttime, endtime, playtime, goals, assists, yc, rc,
                                        penalties, autogoals, player_link, bd_date, bd_strmonth, player_country, 'Штаб', team1_name,
                                        id_team1, match_link, referee_name, stadium] + team1_stats
                    n += 1

                team2_coach = double_coaches.find('td', class_='summary-vertical fr')
                if team2_coach.a is not None:
                    role, number, name, shortname, id_player, starttime, endtime, playtime, goals, \
                    assists, yc, rc, penalties, autogoals, player_link, bd_date, bd_strmonth, player_country = \
                    get_coachinfo(team2_coach, players_dict, playersinfo_dict)

                    df_match.loc[n] = [country, tournament, season, tour, month, match_name, team1_name, score1, score2,
                                        team2_name, 'Гости', result1, id_match, 2*id_match, role, number, name, shortname,
                                        id_player, starttime, endtime, playtime, goals, assists, yc, rc,
                                        penalties, autogoals, player_link, bd_date, bd_strmonth, player_country, 'Штаб', team2_name,
                                        id_team2, match_link, referee_name, stadium] + team2_stats
                    n += 1

                df = df.append(df_match, ignore_index=True)
                id_match += 1

                break

            except (ConnectionError, TimeoutError):
                time.sleep(10)

    #df['гол'] += df['пенальти']

    for _, player_auto in df[df['автогол'] >= 1].iterrows():
        autogoals = player_auto['автогол']
        id_result = player_auto['id_результата']
        if id_result % 2 == 0:
            id_result -= 1
        else:
            id_result += 1

        player_antiauto = df.loc[(df['id_результата'] == id_result) & (df['амплуа'] == 'Нападающий')].iloc[0].name
        df.at[player_antiauto, 'гол'] += 1
        
    return df

In [6]:
driver = webdriver.Firefox()
flashscore = 'https://www.flashscore.ru'

In [18]:
country = 'Англия'
tournament = 'Премьер-лига'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/england/premier-league/results', '/football/england/premier-league-2019-2020/results',
       '/football/england/premier-league-2018-2019/results', '/football/england/premier-league-2017-2018/results',
       '/football/england/premier-league-2016-2017/results', '/football/england/premier-league-2015-2016/results'][::-1]

In [52]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

100%|██████████████████████████████████████████████████████████████████████████████| 427/427 [1:36:15<00:00, 13.53s/it]


FileCreateError: [Errno 2] No such file or directory: 'Англия_2015/2016.xlsx'

In [19]:
country = 'Германия'
tournament = 'Бундеслига'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/germany/bundesliga/results', '/football/germany/bundesliga-2019-2020/results',
       '/football/germany/bundesliga-2018-2019/results', '/football/germany/bundesliga-2017-2018/results',
       '/football/germany/bundesliga-2016-2017/results', '/football/germany/bundesliga-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [20]:
country = 'Италия'
tournament = 'Серия А'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/italy/serie-a/results', '/football/italy/serie-a-2019-2020/results',
       '/football/italy/serie-a-2018-2019/results', '/football/italy/serie-a-2017-2018/results',
       '/football/italy/serie-a-2016-2017/results', '/football/italy/serie-a-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
country = 'Россия'
tournament = 'Премьер-лига'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/russia/premier-league/results', '/football/russia/premier-league-2019-2020/results',
       '/football/russia/premier-league-2018-2019/results', '/football/russia/premier-league-2017-2018/results',
       '/football/russia/premier-league-2016-2017/results', '/football/russia/premier-league-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
country = 'Украина'
tournament = 'Премьер-лига'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/ukraine/premier-league/results', '/football/ukraine/premier-league-2019-2020/results',
       '/football/ukraine/premier-league-2018-2019/results', '/football/ukraine/premier-league-2017-2018/results',
       '/football/ukraine/premier-league-2016-2017/results', '/football/ukraine/premier-league-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
country = 'Франция'
tournament = 'Первая лига'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/france/ligue-1/results', '/football/france/ligue-1-2018-2019/results',
       '/football/france/ligue-1-2018-2019/results', '/football/france/ligue-1-2017-2018/results',
       '/football/france/ligue-1-2016-2017/results', '/football/france/ligue-1-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
country = 'Европа'
tournament = 'Лига чемпионов'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/europe/champions-league/results', '/football/europe/champions-league-2019-2020/results',
       '/football/europe/champions-league-2018-2019/results', '/football/europe/champions-league-2017-2018/results',
       '/football/europe/champions-league-2016-2017/results', '/football/europe/champions-league-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
country = 'Европа'
tournament = 'Лига Европы'
seasons = ['2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016'][::-1]
urls = ['/football/europe/europa-league/results', '/football/europe/europa-league-2019-2020/results',
       '/football/europe/europa-league-2018-2019/results', '/football/europe/europa-league-2017-2018/results',
       '/football/europe/europa-league-2016-2017/results', '/football/europe/europa-league-2015-2016/results'][::-1]

In [ ]:
for season, url in zip(seasons, urls):
    excel = country + '_' + season + '.xlsx'
    df_old = pd.read_excel(excel).iloc[:, 1:] if excel in os.listdir() else None
    
    df = parse(season, url, driver=driver, df_old=df_old)
    df.to_excel(excel)

In [ ]:
driver.close()